# Embeddings  [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1VGqw8yMPsFmlH5IHv8Vca31OVaikcgrh?usp=sharing)

## Word2Vec

Векторные модели, которые мы рассматривали до этого (tf-idf, BOW), условно называются *счётными*. Они основываются на том, что так или иначе "считают" слова и их соседей, и на основе этого строят вектора для слов. 

Другой класс моделей, который более повсевмёстно распространён на сегодняшний день, называется *предсказательными* (или *нейронными*) моделями. Идея этих моделей заключается в использовании нейросетевых архитектур, которые "предсказывают" (а не считают) соседей слов. Одной из самых известных таких моделей является word2vec. Технология основана на нейронной сети, предсказывающей вероятность встретить слово в заданном контексте. Этот инструмент был разработан группой исследователей Google в 2013 году, руководителем проекта был Томаш Миколов (сейчас работает в Facebook). Вот две самые главные статьи:

* [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)
* [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546)


Полученные таким образом вектора называются *Distributed Representations of Words (распределенными представлениями слов)*, или **эмбеддингами**.


### Как это обучается?
Мы задаём вектор для каждого слова с помощью матрицы $w$ и вектор контекста с помощью матрицы $W$. По сути, word2vec является обобщающим названием для двух архитектур Skip-Gram и Continuous Bag-Of-Words (CBOW).  

**CBOW** предсказывает текущее слово, исходя из окружающего его контекста. 

**Skip-gram**, наоборот, использует текущее слово, чтобы предугадывать окружающие его слова. 

### Как это работает?
Word2vec принимает большой текстовый корпус в качестве входных данных и сопоставляет каждому слову вектор, выдавая координаты слов на выходе. Сначала он создает словарь, «обучаясь» на входных текстовых данных, а затем вычисляет векторное представление слов. Векторное представление основывается на контекстной близости: слова, встречающиеся в тексте рядом с одинаковыми словами (а следовательно, согласно дистрибутивной гипотезе, имеющие схожий смысл), в векторном представлении будут иметь близкие координаты векторов-слов. Для вычисления близости слов используется косинусное расстояние между их векторами.


С помощью дистрибутивных векторных моделей можно строить семантические пропорции (они же аналогии) и решать примеры:

* *король: мужчина = королева: женщина* 
 $\Rightarrow$ 
* *король - мужчина + женщина = королева*

![w2v](https://cdn-images-1.medium.com/max/2600/1*sXNXYfAqfLUeiDXPCo130w.png)

### Проблемы
Невозможно установить тип семантических отношений между словами: синонимы, антонимы и т.д. будут одинаково близки, потому что обычно употребляются в схожих контекстах. Поэтому близкие в векторном пространстве слова называют *семантическими ассоциатами*. Это значит, что они семантически связаны, но как именно — непонятно.


### RusVectōrēs


На сайте [RusVectōrēs](https://rusvectores.org/ru/) собраны предобученные на различных данных модели для русского языка, а также можно поискать наиболее близкие слова к заданному, посчитать семантическую близость нескольких слов и порешать примеры с помощью «калькулятора семантической близости».


Для других языков также можно найти предобученные модели — например, модели [fastText](https://fasttext.cc/docs/en/english-vectors.html) и [GloVe](https://nlp.stanford.edu/projects/glove/) (о них чуть дальше).

### Визуализация
А [вот тут](https://projector.tensorflow.org/) есть хорошая визуализация для английского.

## Gensim

Использовать предобученную модель эмбеддингов или обучить свою можно с помощью библиотеки `gensim`. Вот [ее документация](https://radimrehurek.com/gensim/models/word2vec.html).

### Как использовать готовую модель

Модели word2vec бывают разных форматов:

* .vec.gz — обычный файл
* .bin.gz — бинарник

Загружаются они с помощью одного и того же класса `KeyedVectors`, меняется только параметр `binary` у функции `load_word2vec_format`. 

Если же эмбеддинги обучены **не** с помощью word2vec, то для загрузки нужно использовать функцию `load`. Т.е. для загрузки предобученных эмбеддингов *glove, fasttext, bpe* и любых других нужна именно она.

Скачаем с RusVectōrēs модель для русского языка. 

In [1]:
# !pip install gensim
# !pip install bs4

In [2]:
import re
import numpy as np
import os
import zipfile
import gensim
import logging
import nltk.data 
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from gensim.models import word2vec
from gensim.models import KeyedVectors  # module provides a way to work with pre-trained word embeddings
from nltk.tokenize import sent_tokenize, RegexpTokenizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')

from beholder import print_methods, call_methods

[nltk_data] Downloading package punkt to /Users/velo1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Loading the pre-trained model ['ruscorpora_upos_cbow_300_20_2019'](https://rusvectores.org/ru/models/)

In [3]:
# !wget http://vectors.nlpl.eu/repository/20/180.zip
# local NLP folder
nlp_dir ='/Users/velo1/SynologyDrive/GIT_syno/data/NLP/'
permanent_id, file_name = ('ruscorpora_upos_cbow_300_20_2019', '180.zip')

# Check if the target directory already exists (indicating unzipping has been done)
target_dir = nlp_dir + permanent_id + '/'
if not os.path.exists(target_dir):
    # Create the directory if it doesn't exist
    os.makedirs(target_dir)

    # Create a ZipFile object
    with zipfile.ZipFile(nlp_dir + file_name, 'r') as zip_ref:
        # Extract all the files to the specified directory
        zip_ref.extractall(target_dir)

# load the word2vec model
filename = target_dir + 'model.bin'

model_ru = KeyedVectors.load_word2vec_format(filename, binary=True)

In [4]:
# Retrieve a URL into a temporary location on disk.
# urllib.request.urlretrieve(" http://rusvectores.org/static/models/rusvectores2/ruscorpora_mystem_cbow_300_2_2015.bin.gz", "ruscorpora_mystem_cbow_300_2_2015.bin.gz")
# urllib.request.urlretrieve("http://vectors.nlpl.eu/repository/20/220.zip", "ruscorpora_upos_cbow_300_20_2019.zip")

In [5]:
# model_path = 'http://vectors.nlpl.eu/repository/20/180.zip'

# model_ru = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)
# model_ru = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=False)

In [6]:
def tag_mystem2upos(text=["Текст нужно передать функции в виде list!"]) -> list:
    '''Converts POS-tagging from mystem format to UPOS format.'''
    # "https://raw.githubusercontent.com/akutuzov/universal-pos-tags/4653e8a9154e93fe2f417c7fdb7a357b7d6ce333/ru-rnc.map"

    mapping = {
        "A": "ADJ",
        "ADV": "ADV",
        "ADVPRO": "ADV",
        "ANUM": "ADJ",
        "APRO": "DET",
        "COM": "ADJ",
        "CONJ": "SCONJ",
        "INTJ": "INTJ",
        "NONLEX": "X",
        "NUM": "NUM",
        "PART": "PART",
        "PR": "ADP",
        "S": "NOUN",
        "SPRO": "PRON",
        "UNKN": "X",    # X for all unknown tags
        "V": "VERB",
    }
    tagged = []
    for w in text:
        try:
            pos = w.split("_")[1].strip()
            if pos in mapping:
                pos = mapping[pos]  # здесь мы конвертируем тэги
            else:
                pos = "X"  # на случай, если попадется тэг, которого нет в маппинге
            tagged.append(w.split("_")[0] + "_" + pos)
        except KeyError:
            continue  # я здесь пропускаю знаки препинания, но вы можете поступить по-другому

    return tagged

In [7]:
tag_mystem2upos(['мама_S', 'мыть_V', 'рама_S', 'error_e']) 
tag_mystem2upos(['error_e']) 

['error_X']

Возьмем несколько слов для примера:

In [8]:
words = ['день_S', 'ночь_S', 'человек_S', 'семантика_S', 'биткоин_S']
tag_mystem2upos(words)

['день_NOUN', 'ночь_NOUN', 'человек_NOUN', 'семантика_NOUN', 'биткоин_NOUN']

Частеречные тэги нужны, поскольку это специфика скачанной модели - она была натренирована на словах, аннотированных их частями речи (и лемматизированных). **NB!** В названиях моделей на `rusvectores` указано, какой тегсет они используют (mystem, upos и т.д.)

Попросим у модели 10 ближайших соседей для каждого слова и коэффициент косинусной близости для каждого:

In [9]:
for w in words:
    # convert POS-tagging from mystem format to UPOS format
    word = tag_mystem2upos(w.split())[0] # index 0 to get string from list
    # есть ли слово в модели? 
    if word in model_ru:
        # смотрим на вектор слова (его размерность 300, смотрим на первые 10 чисел)
        print(f'{word} shape: {model_ru[word].shape}. First 10 values: {model_ru[word][:10]}')
        # выдаем 10 ближайших соседей слова:
        print(f'\n10 nearest neighbours of "{word}":')
        
        for word, sim in model_ru.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(word, ': ', sim)
        print('\n')
    else:
        # Увы!
        print('Увы, слова "%s" нет в модели!' % word)

день_NOUN shape: (300,). First 10 values: [ 1.805067   -0.877623   -1.0102742   2.8518744  -0.43311968 -3.7207692
 -3.4317713  -0.7634762  -4.9961104  -1.1313324 ]

10 nearest neighbours of "день_NOUN":
неделя_NOUN :  0.7375996112823486
день_PROPN :  0.706766664981842
месяц_NOUN :  0.7037326097488403
час_NOUN :  0.6643950939178467
утро_NOUN :  0.6526744961738586
вечер_NOUN :  0.6038411259651184
сутки_NOUN :  0.5923080444335938
воскресенье_NOUN :  0.5842781066894531
полдень_NOUN :  0.5743688344955444
суббота_NOUN :  0.5345946550369263


ночь_NOUN shape: (300,). First 10 values: [-0.10776415  0.32673436  0.52870405  2.1667976   0.7689093  -2.4214501
 -1.4222336  -2.972895    0.18769576 -0.05231643]

10 nearest neighbours of "ночь_NOUN":
ночь_PROPN :  0.8310787081718445
вечер_NOUN :  0.7183678150177002
рассвет_NOUN :  0.6965947151184082
ночи_NOUN :  0.692021906375885
полночь_NOUN :  0.6704976558685303
ночь_VERB :  0.6615265011787415
утро_NOUN :  0.6263936161994934
ночной_ADJ :  0.60247093

### Находим косинусную близость пары слов:

In [10]:
list_ = tag_mystem2upos(['человек_S', 'обезьяна_S'])
print(model_ru.similarity(*list_))

0.22025342


Что получится, если вычесть из пиццы Италию и прибавить Германию?

* positive — вектора, которые мы складываем
* negative — вектора, которые вычитаем

In [11]:
print(model_ru.most_similar(positive=['пицца_NOUN', 'германия_PROPN'], negative=['италия_PROPN'])[:2])

[('чипсы_NOUN', 0.5982315540313721), ('гамбургер_NOUN', 0.5845527052879333)]


### Find out which word doesn't go with the others

In [12]:
model_ru.doesnt_match(tag_mystem2upos(['пицца_S', 'пельмень_S', 'хот-дог_S', 'ананас_S']))

'ананас_NOUN'

In [13]:
meths = print_methods(model_ru)
# call_methods(model_ru)

59 methods for (<class 'gensim.models.keyedvectors.KeyedVectors'>): 
 ['add_lifecycle_event', 'add_vector', 'add_vectors', 'allocate_vecattrs', 'closer_than', 'cosine_similarities', 'distance', 'distances', 'doesnt_match', 'evaluate_word_analogies', 'evaluate_word_pairs', 'expandos', 'fill_norms', 'get_index', 'get_mean_vector', 'get_normed_vectors', 'get_vecattr', 'get_vector', 'has_index_for', 'index2entity', 'index2word', 'index_to_key', 'init_sims', 'intersect_word2vec_format', 'key_to_index', 'lifecycle_events', 'load', 'load_word2vec_format', 'log_accuracy', 'log_evaluate_word_pairs', 'mapfile_path', 'most_similar', 'most_similar_cosmul', 'most_similar_to_given', 'n_similarity', 'next_index', 'norms', 'rank', 'rank_by_centrality', 'relative_cosine_similarity', 'resize_vectors', 'save', 'save_word2vec_format', 'set_vecattr', 'similar_by_key', 'similar_by_vector', 'similar_by_word', 'similarity', 'similarity_unseen_docs', 'sort_by_descending_frequency', 'unit_normalize_all', 'vecto

**Упражнения для разминки**

Найдите пример многозначного слова, для которого в топ-10 (метод `most_similar`) похожих на него слов входят слова связанные с разными значениями:

In [14]:
model_ru.most_similar(positive=['знак_NOUN'], topn=10)

[('знак_ADV', 0.6484130620956421),
 ('знак_PROPN', 0.5194986462593079),
 ('символ_NOUN', 0.47708243131637573),
 ('значок_NOUN', 0.4415062963962555),
 ('эмблема_NOUN', 0.4384552240371704),
 ('символический_ADJ', 0.4278099834918976),
 ('надпись_NOUN', 0.4255180358886719),
 ('нарукавный_ADJ', 0.42526695132255554),
 ('кабалистический_ADJ', 0.4215017259120941),
 ('обозначение_NOUN', 0.4212318956851959)]

In [15]:
model_ru.most_similar(positive=['штопор_NOUN'], topn=10)

[('руль_VERB', 0.5641716718673706),
 ('вираж_NOUN', 0.5314818024635315),
 ('пропеллер_NOUN', 0.5066705942153931),
 ('пикирование_NOUN', 0.49755290150642395),
 ('стефановский_PROPN', 0.49500828981399536),
 ('арцеулов_PROPN', 0.4792757034301758),
 ('парашют_VERB', 0.47498655319213867),
 ('траектория_NOUN', 0.4714280068874359),
 ('винт_NOUN', 0.4686664342880249),
 ('спираль_NOUN', 0.467825323343277)]

По аналогии с Италия -- пицца, Сибирь -- пельмень, придумайте похожую связку слов для проверки: 

In [16]:
model_ru.most_similar(positive=['птица_NOUN', 'нос_NOUN'], negative=['человек_NOUN'])[:5][:1]

[('клюв_NOUN', 0.5999110341072083)]

Приведите пример трех слов w1, w2, w3, таких, что w1 и w2 являются синонимами, w1 и w3 являются антонимами, но при этом, similarity(w1, w2) < similarity(w1, w3).

In [17]:
w1, w2, w3 = 'веселый_ADJ', 'улыбчивый_ADJ', 'грустный_ADJ'
model_ru.similarity(w1, w2) < model_ru.similarity(w1, w3)

True

### Задание

Напишите функцию, которая принимает на вход предложение, и заменяет случайное существительное в нём на "ассоциат" -- ближайшее к нему слово из модели word2vec.

NB: для этого вам понадобится морфологический анализатор. Советую использовать pymorphy (мы кратко говорили про него на прошлом семинаре).

 как пользоваться pymorphy:

In [18]:
# !pip install pymorphy2
from pymorphy2 import MorphAnalyzer
analyser = MorphAnalyzer()

In [19]:
# разобрать слово (в данном случае возможно два разбора, поэтому получаем список из двух элементов)
result = analyser.parse('слово')
result

[Parse(word='слово', tag=OpencorporaTag('NOUN,inan,neut sing,nomn'), normal_form='слово', score=0.615384, methods_stack=((<DictionaryAnalyzer>, 'слово', 54, 0),)),
 Parse(word='слово', tag=OpencorporaTag('NOUN,inan,neut sing,accs'), normal_form='слово', score=0.384615, methods_stack=((<DictionaryAnalyzer>, 'слово', 54, 3),))]

In [20]:
# достать часть речи
result[0].tag.POS

'NOUN'

In [21]:
# поставить в дательный падеж
result[0].inflect(frozenset(['datv'])).word

'слову'

Ваша функция (для простоты можно не пытаться поставить слово в "нужную" форму и ограничиться именительным падежом):

In [22]:
# import random
import pymorphy2
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize


# Initialize pymorphy2
morph = pymorphy2.MorphAnalyzer()


In [23]:
def inflect_word_to_case(word, target_word):
    '''Inflect a word to the case of another word'''
    target_parsed = morph.parse(target_word)[0]
    word_parsed = morph.parse(word)[0]
    inflected_form = word_parsed.inflect({target_parsed.tag.case})
    
    if inflected_form:
        return inflected_form.word
    else:
        return word  # Return the original word if inflection is not possible


def replace_noun_with_associate(sentence, word2vec_model, sim_rank=1):
    """Replace a random noun in the sentence with a similar word from Word2Vec"""

    words = word_tokenize(sentence)    # Split sentence into list of words
    new_words = []

    for word in words:  # Iterate over each word of the sentence
        parsed = analyser.parse(word)[0]    # Parse the word with pymorphy2
        if 'NOUN' in parsed.tag:        # If the word is a noun
            # Find the most similar word from the vocabulary
            word2replace = parsed.normal_form + '_' + 'NOUN'
            similar_words = word2vec_model.most_similar(positive=[word2replace], topn=sim_rank)
            if similar_words:   # If there's a similar word, replace it
                closest_word = similar_words[sim_rank-1][0].rsplit('_', 1)[0]  # Remove the UPOS tag 
                closest_word = inflect_word_to_case(closest_word, word)  # Inflect the word to the case of the original word
                new_words.append(closest_word)
            else:
                new_words.append(word)
        else:
            new_words.append(word)

    return ' '.join(new_words)

# Example sentence
input_sentence = "Кот преследовал мышь по всему двору  до вечера."

# Replace a random noun with a similar word from Word2Vec
output_sentence = replace_noun_with_associate(input_sentence, model_ru)
print("Input Sentence:", input_sentence)
print("Output Sentence:", output_sentence)

Input Sentence: Кот преследовал мышь по всему двору  до вечера.
Output Sentence: кошка преследовал крысу по всему сараю до утра .


In [24]:
# Example sentence
input_sentence = "Ехал медведь на велосипеде и упал в лужу.\n"
print(f"Input Sentence: {input_sentence}")
# Replace a random noun with a similar word from Word2Vec
for sim_rank in range(1, 11):
    output_sentence = replace_noun_with_associate(input_sentence, model_ru, sim_rank=sim_rank)
    print(f"Output Sentence: {output_sentence}") 

Input Sentence: Ехал медведь на велосипеде и упал в лужу.

Output Sentence: Ехал медведь на мотоцикле и упал в лужу .
Output Sentence: Ехал медведь на трехколёсном и упал в лужицу .
Output Sentence: Ехал зверь на мотороллере и упал в жижу .
Output Sentence: Ехал волк на самокате и упал в грязь .
Output Sentence: Ехал медведица на велосипедисте и упал в асфальт .
Output Sentence: Ехал барсук на велосипедном и упал в мокрого .
Output Sentence: Ехал тигр на мопеде и упал в тротуар .
Output Sentence: Ехал заяц на автомобиле и упал в канаву .
Output Sentence: Ехал кабан на коньке и упал в выбоину .
Output Sentence: Ехал медвежонок на двухколёсном и упал в колдобину .


## Как обучить свою модель

В качестве обучающих данных возьмем размеченные и неразмеченные отзывы о фильмах (датасет взят с Kaggle).

In [25]:
# !wget https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/unlabeledTrainData.tsv

data = pd.read_csv("unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

print(len(data))
data.head()

50000


,id,review
0,"""9999_0""","""Watching Time Chasers, it obvious that it was..."
1,"""45057_0""","""I saw this film about 20 years ago and rememb..."
2,"""15561_0""","""Minor Spoilers<br /><br />In New York, Joan B..."
3,"""7161_0""","""I went to see this film with a great deal of ..."
4,"""43971_0""","""Yes, I agree with everyone on this site this ..."


Убираем из данных ссылки, html-разметку и небуквенные символы, а затем приводим все к нижнему регистру и токенизируем. На выходе получается массив из предложений, каждое из которых представляет собой массив слов. Здесь используется токенизатор из библиотеки `nltk`. 

In [26]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [27]:
def review_to_wordlist(review, remove_stopwords=False ):
    # убираем ссылки
    review_text = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", " ", review)
    # достаем сам текст
#     review_text = BeautifulSoup(review_text, "lxml").get_text()
    # оставляем только буквенные символы
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    # приводим к нижнему регистру и разбиваем на слова по символу пробела
    words = review_text.lower().split()
    if remove_stopwords: # убираем стоп-слова
        stops = stopwords.words("english")
        words = [w for w in words if not w in stops]
    return(words)

def review_to_sentences(review, tokenizer, remove_stopwords=False):
    # разбиваем обзор на предложения
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    # применяем предыдущую функцию к каждому предложению
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

In [28]:
from tqdm import tqdm

### Clean the text

In [29]:
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

sentences = []  

print("Parsing sentences from training set...")
for review in tqdm(data["review"]):
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set...


100%|██████████| 50000/50000 [00:17<00:00, 2848.52it/s]


In [30]:
print(len(sentences))
print(sentences[0])

529416
['watching', 'time', 'chasers', 'it', 'obvious', 'that', 'it', 'was', 'made', 'by', 'a', 'bunch', 'of', 'friends']


In [31]:
# это понадобится нам позже

with open('clean_text.txt', 'w') as f:
    # for s in sentences[:5000]:
    for s in sentences:        
        f.write(' '.join(s))
        f.write('\n')

Обучаем и сохраняем модель. 


Основные параметры:
* данные должны быть итерируемым объектом 
* size — размер вектора, 
* window — размер окна наблюдения,
* min_count — мин. частотность слова в корпусе,
* sg — используемый алгоритм обучения (0 — CBOW, 1 — Skip-gram),
* sample — порог для downsampling'a высокочастотных слов,
* workers — количество потоков,
* alpha — learning rate,
* iter — количество итераций,
* max_vocab_size — позволяет выставить ограничение по памяти при создании словаря (т.е. если ограничение превышается, то низкочастотные слова будут выбрасываться). Для сравнения: 10 млн слов = 1Гб RAM.

**NB!** Обратите внимание, что тренировка модели не включает препроцессинг! Это значит, что избавляться от пунктуации, приводить слова к нижнему регистру, лемматизировать их, проставлять частеречные теги придется до тренировки модели (если, конечно, это необходимо для вашей задачи). Т.е. в каком виде слова будут в исходном тексте, в таком они будут и в модели.

In [32]:
print("Training model...")

%time model_en = word2vec.Word2Vec(sentences, workers=4, vector_size=300, min_count=10, window=10, sample=1e-3)

Training model...
CPU times: user 2min 54s, sys: 2.07 s, total: 2min 56s
Wall time: 48.5 s


Смотрим, сколько в модели слов.

In [33]:
print(len(model_en.wv.key_to_index))

28316


Попробуем оценить модель вручную, порешав примеры. Несколько дано ниже, попробуйте придумать свои.

In [34]:
print(model_en.wv.most_similar(positive=["woman", "actor"], negative=["man"], topn=1))
print(model_en.wv.most_similar(positive=["dogs", "man"], negative=["dog"], topn=1))

print(model_en.wv.most_similar("usa", topn=3))

print(model_en.wv.doesnt_match("comedy thriller western novel".split()))

[('actress', 0.7513849139213562)]
[('men', 0.6290287971496582)]
[('europe', 0.7354639768600464), ('uk', 0.7176207304000854), ('germany', 0.7155715823173523)]
novel


### Как дообучить существующую модель

При тренировке модели "с нуля" веса инициализируются случайно, однако можно использовать для инициализации векторов веса из предобученной модели, таким образом как бы дообучая ее.

Сначала посмотрим близость какой-нибудь пары слов в имеющейся модели, чтобы потом сравнить результат с дообученной.

In [35]:
model_en.wv.similarity('lion', 'rabbit')

0.31098974

В качестве дополнительных данных для обучения возьмем английский текст «Алисы в Зазеркалье».

In [36]:
with open("alice.txt", 'r', encoding='utf-8') as f:
    text = f.read()

# убираем переносы строк, токенизируем текст
text = re.sub('\n', ' ', text)
sents = sent_tokenize(text)

# убираем всю пунктуацию и делим текст на слова по пробелу
punct = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~„“«»†*—/\-‘’'
clean_sents = []
for sent in sents:
    s = [w.lower().strip(punct) for w in sent.split()]
    clean_sents.append(s)
    
print(clean_sents[:2])

[['through', 'the', 'looking-glass', 'by', 'lewis', 'carroll', 'chapter', 'i', 'looking-glass', 'house', 'one', 'thing', 'was', 'certain', 'that', 'the', 'white', 'kitten', 'had', 'had', 'nothing', 'to', 'do', 'with', 'it', '', 'it', 'was', 'the', 'black', 'kitten’s', 'fault', 'entirely'], ['for', 'the', 'white', 'kitten', 'had', 'been', 'having', 'its', 'face', 'washed', 'by', 'the', 'old', 'cat', 'for', 'the', 'last', 'quarter', 'of', 'an', 'hour', 'and', 'bearing', 'it', 'pretty', 'well', 'considering', 'so', 'you', 'see', 'that', 'it', 'couldn’t', 'have', 'had', 'any', 'hand', 'in', 'the', 'mischief']]


Чтобы дообучить модель, надо сначала ее сохранить, а потом загрузить. Все параметры тренировки (размер вектора, мин. частота слова и т.п.) будут взяты из загруженной модели, т.е. задать их заново нельзя.

**NB!** Дообучить можно только полную модель, а `KeyedVectors` — нельзя. Поэтому сохранять модель нужно в соотвествующем формате. Подробнее о разнице [вот тут](https://radimrehurek.com/gensim/models/keyedvectors.html).

In [37]:
model_path = "movie_reviews.model" # save the model with different name

print("Saving model...")
model_en.save(model_path)

Saving model...


In [38]:
model = word2vec.Word2Vec.load(model_path) # load model

model.build_vocab(clean_sents, update=True) # update vocabulary
model.train(clean_sents, total_examples=model.corpus_count, epochs=5)   # train model

(96943, 150225)

Лев и кролик стали чуть ближе друг к другу!

In [39]:
model.wv.similarity('lion', 'rabbit') > model_en.wv.similarity('lion', 'rabbit')

True

Можно нормализовать вектора, тогда модель будет занимать меньше RAM. Однако после этого её нельзя дотренировывать. Здесь используется L2-нормализация: вектора нормализуются так, что если сложить квадраты всех элементов вектора, в сумме получится 1. 

Кроме того, сохраним не полные вектора, а `KeyedVectors`.

In [40]:
model.init_sims(replace=True)   # normalize vectors
model_path = "movies_alice.bin"

print("Saving model...")
# save the first model as binary file
model_en.wv.save_word2vec_format(model_path, binary=True)

/var/folders/qw/qq8zb5hx0w9cgzgkf9_v2sh00000gq/T/ipykernel_82450/444224005.py:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)   # normalize vectors


Saving model...


## Оценка

Это, конечно, хорошо, но как понять, какая модель лучше? Или вот, например, я сделал свою модель, а как понять, насколько она хорошая?

Для этого существуют специальные датасеты для оценки качества дистрибутивных моделей. Основных два: один измеряет точность решения задач на аналогии (про Россию и пельмени), а второй используется для оценки коэффициента семантической близости. 

### Word Similarity

Этот метод заключается в том, чтобы оценить, насколько представления о семантической близости слов в модели соотносятся с "представлениями" людей.

| слово 1    | слово 2    | близость | 
|------------|------------|----------|
| кошка      | собака     | 0.7      |  
| чашка      | кружка     | 0.9      |       

Для каждой пары слов из заранее заданного датасета мы можем посчитать косинусное расстояние, и получить список таких значений близости. При этом у нас уже есть список значений близостей, сделанный людьми. Мы можем сравнить эти два списка и понять, насколько они похожи (например, посчитав корреляцию). Эта мера схожести должна говорить о том, насколько модель хорошо моделирует расстояния до слова.

### Аналогии

Другая популярная задача для "внутренней" оценки называется задачей поиска аналогий. Как мы уже разбирали выше, с помощью простых арифметических операций мы можем модифицировать значение слова. Если заранее собрать набор слов-модификаторов, а также слов, которые мы хотим получить в результаты модификации, то на основе подсчёта количества "попаданий" в желаемое слово мы можем оценить, насколько хорошо работает модель.

В качестве слов-модификаторов мы можем использовать семантические аналогии. Скажем, если у нас есть некоторое отношение "страна-столица", то для оценки модели мы можем использовать пары наподобие "Россия-Москва", "Норвегия-Осло", и т.д. Датасет будет выглядеть следующм образом:

| слово 1    | слово 2    | отношение     | 
|------------|------------|---------------|
| Россия     | Москва     | страна-столица|  
| Норвегия   | Осло       | страна-столица|

Рассматривая случайные две пары из этого набора, мы хотим, имея триплет (Россия, Москва, Норвегия) хотим получить слово "Осло", т.е. найти такое слово, которое будет находиться в том же отношении со словом "Норвегия", как "Россия" находится с Москвой. 

Датасеты для русского языка можно скачать на странице с моделями на RusVectores.

In [41]:
# ! wget https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/evaluation/ru_analogy_tagged.txt
!head ru_analogy_tagged.txt

: capital-common-countries
афины_S греция_S багдад_S ирак_S
афины_S греция_S бангкок_S таиланд_S
афины_S греция_S пекин_S китай_S
афины_S греция_S берлин_S германия_S
афины_S греция_S берн_S швейцария_S
афины_S греция_S каир_S египет_S
афины_S греция_S канберра_S австралия_S
афины_S греция_S ханой_S вьетнам_S
афины_S греция_S гавана_S куба_S


## FastText

FastText использует не только эмбеддинги слов, но и эмбеддинги n-грам. В корпусе каждое слово автоматически представляется в виде набора символьных n-грамм. Скажем, если мы установим n=3, то вектор для слова "where" будет представлен суммой векторов следующих триграм: "<wh", "whe", "her", "ere", "re>" (где "<" и ">" символы, обозначающие начало и конец слова). Благодаря этому мы можем также получать вектора для слов, отсутствуюших в словаре, а также эффективно работать с текстами, содержащими ошибки и опечатки.

* [Статья](https://aclweb.org/anthology/Q17-1010)
* [Сайт](https://fasttext.cc/)
* [Тьюториал](https://fasttext.cc/docs/en/support.html)
* [Вектора для 157 языков](https://fasttext.cc/docs/en/crawl-vectors.html)
* [Вектора, обученные на википедии](https://fasttext.cc/docs/en/pretrained-vectors.html) (отдельно для 294 разных языков)
* [Репозиторий](https://github.com/facebookresearch/fasttext)

Есть библиотека `fasttext` для питона (с готовыми моделями можно работать и через `gensim`).

In [42]:
# ! git clone https://github.com/facebookresearch/fastText.git
# ! pip3 install fastText/.
import fasttext

# Train an unsupervised model and return a model object. 
# Input must be a filepath. The input text does not need to be tokenized as per 
# the tokenize function, but it must be preprocessed and encoded as UTF-8.
ft_model = fasttext.train_unsupervised('clean_text.txt', minn=3, maxn=4, dim=300)

Read 12M words
Number of words:  39721
Number of labels: 0
Progress: 100.0% words/sec/thread:   21284 lr:  0.000000 avg.loss:  2.201756 ETA:   0h 0m 0s0h 4m23s 11.3% words/sec/thread:   14613 lr:  0.044337 avg.loss:  2.242726 ETA:   0h 4m15sm 2s 29.4% words/sec/thread:   15932 lr:  0.035322 avg.loss:  2.242181 ETA:   0h 3m 6s 31.8% words/sec/thread:   16147 lr:  0.034123 avg.loss:  2.241171 ETA:   0h 2m57s 34.0% words/sec/thread:   16084 lr:  0.032984 avg.loss:  2.239221 ETA:   0h 2m52s  16210 lr:  0.029984 avg.loss:  2.235671 ETA:   0h 2m35s 40.3% words/sec/thread:   16222 lr:  0.029873 avg.loss:  2.235320 ETA:   0h 2m34s 40.6% words/sec/thread:   16231 lr:  0.029705 avg.loss:  2.235434 ETA:   0h 2m33s


In [43]:
ft_model.get_word_vector("movie")[:10]

array([-0.05911254,  0.02789777, -0.01386697, -0.129203  , -0.00770547,
       -0.01958451,  0.06880166,  0.23435326, -0.07443594, -0.16534059],
      dtype=float32)

In [44]:
ft_model.get_nearest_neighbors('actor')

[(0.6737564206123352, 'actors'),
 (0.5949331521987915, 'tractor'),
 (0.5653902888298035, 'ctor'),
 (0.5475830435752869, 'reactor'),
 (0.53066086769104, 'actress'),
 (0.49728068709373474, 'performance'),
 (0.4859069585800171, 'talented'),
 (0.47997209429740906, 'lector'),
 (0.47709763050079346, 'performer'),
 (0.4746330678462982, 'role')]

In [45]:
# проблема с опечатками решена

ft_model.get_nearest_neighbors('actr')

[(0.7664213180541992, 'actress'),
 (0.7175777554512024, 'actresses'),
 (0.6524512767791748, 'actors'),
 (0.6201873421669006, 'actor'),
 (0.5056630373001099, 'talented'),
 (0.4900783598423004, 'acte'),
 (0.484948992729187, 'acting'),
 (0.47762584686279297, 'performers'),
 (0.47757741808891296, 'juhi'),
 (0.4595927596092224, 'ehsaan')]

In [46]:
# проблема с out of vocabulary словами - тоже

ft_model.get_nearest_neighbors('moviegeek')

[(0.6338699460029602, 'moviegoing'),
 (0.6033056378364563, 'movie'),
 (0.6021876335144043, 'geek'),
 (0.6003331542015076, 'moviegoer'),
 (0.5785683393478394, 'mfj'),
 (0.5781276226043701, 'movies'),
 (0.566527247428894, 'moviegoers'),
 (0.563457190990448, 'sb'),
 (0.5476100444793701, 'hpl'),
 (0.5433463454246521, 'beek')]

In [47]:
df = pd.read_csv('/Users/velo1/SynologyDrive/GIT_syno/data/NLP/tweets_sentiment.csv')

In [48]:
df.head()

,text,label
0,мыс на меня обиделась:(\nя ей даже ничего не с...,negative
1,"аааааааааааааааааааа,не хочу на работу :(",negative
2,"У меня какой-то особенный вид ушей! :D, некото...",positive
3,@simonovkon он неплохой человек в жизни. Я ра...,negative
4,"RT @Darina_Lo: Домааааа\nЕхали на такси, пели ...",positive


In [49]:
len(df)

226834

Запишем полученные данные в формате для обучения классификатора:

In [50]:
import numpy as np
from sklearn.model_selection import train_test_split

In [52]:
X = df.text.tolist()
y = df.label.tolist()

X, y = np.array(X), np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33)
print ("total train examples %s" % len(y_train))
print ("total test examples %s" % len(y_test))

total train examples 151978
total test examples 74856


In [53]:
with open('data.train.txt', 'w+') as outfile:
    # format train data for fasttext classifier
    for i in range(len(X_train)):
        outfile.write('__label__' + y_train[i] + ' '+ X_train[i] + '\n')
    

with open('test.txt', 'w+') as outfile:
    # format test data for fasttext classifier
    for i in range(len(X_test)):
        outfile.write('__label__' + y_test[i] + ' ' + X_test[i] + '\n')

In [54]:
classifier = fasttext.train_supervised('data.train.txt')
result = classifier.test('test.txt')

print('P@1:', result[1])    # Precision at Recall one
print('R@1:', result[2])    # Recall at Precision one
print('Number of examples:', result[0])

Read 2M words
Number of words:  422367
Number of labels: 2
Progress: 100.0% words/sec/thread:  887760 lr:  0.000000 avg.loss:  0.213438 ETA:   0h 0m 0s


P@1: 0.8358982579886716
R@1: 0.8358982579886716
Number of examples: 74856


## Задание

1. Мы будем работать с данными fakenews отсюда: https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv
2. Проведите препроцессинг текста. Разбейте данные на train и test для задачи классификации.
3. Обучите свою модель w2v (или возьмите любую подходящую предобученную модель). Реализуйте функцию для вычисления вектора текста. 
4. Обучите на полученных средних векторах алгоритм классификации.

Бонус: Модифицируйте функцию вычисления среднего вектора: взвешивайте вектора слов соответствующими весами tf-idf.

In [68]:
# !wget https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv
df = pd.read_csv('Constraint_Train.csv')
df.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [69]:
df.shape

(6420, 3)

In [70]:
corpora = [word_tokenize(text.lower()) for text in tqdm(df['tweet'])]

 14%|█▍        | 924/6420 [00:00<00:01, 4457.76it/s]

100%|██████████| 6420/6420 [00:01<00:00, 4930.58it/s]


In [71]:
model_tweets = Word2Vec(corpora, min_count=2, vector_size=200, workers=4, window=10, sg=1) # initialize model
model_tweets

In [72]:
model_tweets.wv.most_similar('president')

[('donald', 0.9530353546142578),
 ('trump', 0.9352554678916931),
 ('obama', 0.9114581942558289),
 ('barack', 0.9011787176132202),
 ('former', 0.8968438506126404),
 ('joe', 0.8864171504974365),
 ('downplayed', 0.8814111351966858),
 ('tweeted', 0.8778067827224731),
 ('pelosi', 0.8752232789993286),
 ('duterte', 0.8705849647521973)]

In [73]:
def get_tweet_embedding(tweet):
    '''Returns the embedding of a text as the SUM of the embeddings of its words.'''
    
    tokens = [word for word in tweet]
    embedding = np.zeros((model_tweets.vector_size,), dtype=np.float32)
    for token in tokens:
        if token in model_tweets.wv:
            embedding += model_tweets.wv[token]
            
    return embedding

In [74]:
features = [get_tweet_embedding(tweet) for tweet in corpora]

In [75]:
features[0].shape, len(features)

((200,), 6420)

In [76]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [77]:
X_train, X_test, y_train, y_test = train_test_split(features, df['label'].to_list(), test_size=0.33, random_state=42)

In [78]:
model = LogisticRegression(max_iter=2000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=2000)

In [79]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print(cm)

              precision    recall  f1-score   support

        fake       0.91      0.93      0.92      1004
        real       0.93      0.92      0.93      1115

    accuracy                           0.92      2119
   macro avg       0.92      0.93      0.92      2119
weighted avg       0.93      0.92      0.92      2119

[[ 932   72]
 [  87 1028]]


In [80]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print(cm)


              precision    recall  f1-score   support

        fake       0.93      0.91      0.92      1004
        real       0.92      0.94      0.93      1115

    accuracy                           0.93      2119
   macro avg       0.93      0.93      0.93      2119
weighted avg       0.93      0.93      0.93      2119

[[ 915   89]
 [  64 1051]]
